In [1]:
%pip install "deepgram-sdk>=3.*,<4.0"
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import asyncio
from dotenv import load_dotenv
from deepgram import DeepgramClient, PrerecordedOptions, FileSource

load_dotenv()

# Load API key from environment variable
API_KEY = os.getenv("5b9c236b2bbc73dbd78e0916d8d98acd6ee2bac4")

# Path to the folder containing audio files
AUDIO_FOLDER = "../Audio"

# Path to the folder where transcription results will be saved
OUTPUT_FOLDER = "/TT"

async def transcribe_audio_file(file_path, deepgram):
    try:
        with open(file_path, "rb") as file:
            buffer_data = file.read()

        payload = {
            "buffer": buffer_data,
        }

        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        response = await deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # Save the response to a JSON file in the output folder
        output_file_path = os.path.join(OUTPUT_FOLDER, os.path.basename(file_path).replace(".wav", ".json"))
        with open(output_file_path, "w") as output_file:
            json.dump(response.to_dict(), output_file, indent=4)

        print(f"Transcription saved to {output_file_path}")
        print("-------------------------------------------------------")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

async def process_audio_files():
    try:
        # Create a Deepgram client using the API key
        deepgram = DeepgramClient(API_KEY)

        # Create the output folder if it doesn't exist
        if not os.path.exists(OUTPUT_FOLDER):
            os.makedirs(OUTPUT_FOLDER)

        # Loop through all files in the audio folder and transcribe each one asynchronously
        tasks = [transcribe_audio_file(os.path.join(AUDIO_FOLDER, filename), deepgram) for filename in os.listdir(AUDIO_FOLDER) if filename.endswith(".wav")]
        
        await asyncio.gather(*tasks)

    except Exception as e:
        print(f"Exception: {e}")

def main():
    loop = asyncio.get_event_loop()
    loop.run_until_complete(process_audio_files())
    loop.close()

if __name__ == "__main__":
    asyncio.run(process_audio_files())



RuntimeError: asyncio.run() cannot be called from a running event loop